# 10 - Instruction Set Architecture (ISA)

## Introduction

The **Instruction Set Architecture (ISA)** defines the language our CPU understands. It specifies:

- What instructions are available
- How instructions are encoded in binary
- What each instruction does

Our 8-bit computer uses **16-bit instructions** with a 4-bit opcode, giving us 16 possible instructions.

## Learning Objectives

1. Understand instruction encoding
2. Learn our 16-instruction set
3. Build instruction encode/decode functions

In [ ]:
import sys
from pathlib import Path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))
sys.path.insert(0, str(project_root))

from typing import List, Dict
print("Setup complete!")

## Instruction Format

Our 16-bit instruction has different formats depending on the instruction type:

**R-type (ALU operations: ADD, SUB, AND, OR, XOR)**:
```mermaid
flowchart LR
    subgraph Instruction[R-type: 16 bits]
        direction LR
        OP["Opcode<br/>15-12"]
        RD["Rd<br/>11-8"]
        RS1["Rs1<br/>7-4"]
        RS2["Rs2<br/>3-0"]
    end
```

**I-type (Memory operations: LOAD, STORE)**:
```mermaid
flowchart LR
    subgraph Instruction[I-type: 16 bits]
        direction LR
        OP["Opcode<br/>15-12"]
        RD["Rd<br/>11-8"]
        ADDR["Address<br/>7-0 (8 bits)"]
    end
```

**J-type (Jumps: JMP, JZ, JNZ)**:
```mermaid
flowchart LR
    subgraph Instruction[J-type: 16 bits]
        direction LR
        OP["Opcode<br/>15-12"]
        UNUSED["unused<br/>11-8"]
        ADDR["Address<br/>7-0 (8 bits)"]
    end
```

**Key Design Decisions:**
- R-type uses 4-bit fields for registers (only 3 bits needed for 8 registers)
- I-type and J-type use 8-bit addresses, allowing access to full 256-byte memory

## Our 16 Instructions

In [ ]:
# Opcode definitions
OPCODES = {
    'NOP':   0b0000,  # No operation
    'LOAD':  0b0001,  # Load from memory
    'STORE': 0b0010,  # Store to memory
    'MOV':   0b0011,  # Copy register
    'ADD':   0b0100,  # Add
    'SUB':   0b0101,  # Subtract
    'AND':   0b0110,  # Bitwise AND
    'OR':    0b0111,  # Bitwise OR
    'XOR':   0b1000,  # Bitwise XOR
    'NOT':   0b1001,  # Bitwise NOT
    'SHL':   0b1010,  # Shift left
    'SHR':   0b1011,  # Shift right
    'JMP':   0b1100,  # Unconditional jump
    'JZ':    0b1101,  # Jump if zero
    'JNZ':   0b1110,  # Jump if not zero
    'HALT':  0b1111,  # Stop execution
}

# Reverse lookup
OPCODE_NAMES = {v: k for k, v in OPCODES.items()}

# Display the instruction set
print("Our 8-bit Computer Instruction Set:")
print("-" * 50)
print(f"{'Opcode':6} {'Binary':6} {'Mnemonic':8} {'Description'}")
print("-" * 50)
descriptions = {
    'NOP': 'No operation',
    'LOAD': 'Rd = Memory[addr]',
    'STORE': 'Memory[addr] = Rs',
    'MOV': 'Rd = Rs',
    'ADD': 'Rd = Rs1 + Rs2',
    'SUB': 'Rd = Rs1 - Rs2',
    'AND': 'Rd = Rs1 & Rs2',
    'OR': 'Rd = Rs1 | Rs2',
    'XOR': 'Rd = Rs1 ^ Rs2',
    'NOT': 'Rd = ~Rs',
    'SHL': 'Rd = Rs << 1',
    'SHR': 'Rd = Rs >> 1',
    'JMP': 'PC = addr',
    'JZ': 'if (Z) PC = addr',
    'JNZ': 'if (!Z) PC = addr',
    'HALT': 'Stop execution',
}
for name, code in OPCODES.items():
    print(f"{code:04b}   {code:2d}     {name:8} {descriptions[name]}")

## Exercise 1: Instruction Encoding

Encode an instruction from its parts into a 16-bit value.

In [ ]:
def int_to_bits_n(value: int, n: int) -> List[int]:
    """Convert integer to n-bit list (LSB first)."""
    return [(value >> i) & 1 for i in range(n)]

def bits_to_int_n(bits: List[int]) -> int:
    """Convert bit list to integer."""
    return sum(bit << i for i, bit in enumerate(bits))

def encode_instruction(opcode: str, rd: int = 0, rs1: int = 0, rs2_imm: int = 0) -> List[int]:
    """
    Encode an instruction as 16 bits.
    
    Args:
        opcode: Instruction name (e.g., 'ADD', 'LOAD')
        rd: Destination register (0-7)
        rs1: Source register 1 (0-7, for R-type)
        rs2_imm: Source register 2 (0-7 for R-type) or address (0-255 for I/J-type)
    
    Returns:
        16-bit instruction (LSB first)
    
    Note:
        - R-type: opcode(4) + rd(4) + rs1(4) + rs2(4)
        - I-type (LOAD/STORE): opcode(4) + rd(4) + addr(8)
        - J-type (JMP/JZ/JNZ): opcode(4) + unused(4) + addr(8)
    """
    # YOUR CODE HERE
    # 1. Get opcode value from OPCODES dict
    # 2. Check instruction type to determine format:
    #    - LOAD/STORE: use 8-bit address in low byte
    #    - JMP/JZ/JNZ: use 8-bit address in low byte
    #    - Others: standard R-type format
    # 3. Return as 16-bit list
    pass

# Test
# ADD R0, R1, R2 should be: opcode=0100, rd=0, rs1=1, rs2=2
instr = encode_instruction('ADD', rd=0, rs1=1, rs2_imm=2)
print(f"ADD R0, R1, R2 = {bits_to_int_n(instr):04X}")

# LOAD R3, 0x10 should be: opcode=0001, rd=3, addr=0x10
instr = encode_instruction('LOAD', rd=3, rs2_imm=0x10)
print(f"LOAD R3, 0x10 = {bits_to_int_n(instr):04X}")

# JMP 0x20 should be: opcode=1100, addr=0x20
instr = encode_instruction('JMP', rs2_imm=0x20)
print(f"JMP 0x20 = {bits_to_int_n(instr):04X}")

# HALT should be: opcode=1111
instr = encode_instruction('HALT')
print(f"HALT = {bits_to_int_n(instr):04X}")

## Exercise 2: Instruction Decoding

Decode a 16-bit instruction into its parts.

In [ ]:
def decode_instruction(instruction: List[int]) -> Dict:
    """
    Decode a 16-bit instruction.
    
    Args:
        instruction: 16-bit instruction (LSB first)
    
    Returns:
        Dictionary with decoded fields:
        - opcode: integer opcode
        - opcode_name: string name (e.g., 'ADD')
        - rd: destination register
        - rs1: source register 1
        - rs2_imm: source register 2 OR 8-bit address
    
    Note:
        For LOAD/STORE/JMP/JZ/JNZ, rs2_imm is 8 bits (address).
        For R-type instructions, rs2_imm is 4 bits (register).
    """
    # YOUR CODE HERE
    # 1. Convert to integer
    # 2. Extract opcode (bits 15-12)
    # 3. Look up opcode name
    # 4. Based on instruction type:
    #    - For I/J-type: extract rd(11-8), addr(7-0)
    #    - For R-type: extract rd(11-8), rs1(7-4), rs2(3-0)
    pass

# Test
# Encode and decode ADD R0, R1, R2
instr = encode_instruction('ADD', rd=0, rs1=1, rs2_imm=2)
decoded = decode_instruction(instr)
print("Decoded ADD R0, R1, R2:")
for key, value in decoded.items():
    print(f"  {key}: {value}")

print()

# Encode and decode LOAD R3, 0x10
instr = encode_instruction('LOAD', rd=3, rs2_imm=0x10)
decoded = decode_instruction(instr)
print("Decoded LOAD R3, 0x10:")
for key, value in decoded.items():
    print(f"  {key}: {value}")

print()

# Encode and decode JMP 0x20
instr = encode_instruction('JMP', rs2_imm=0x20)
decoded = decode_instruction(instr)
print("Decoded JMP 0x20:")
for key, value in decoded.items():
    print(f"  {key}: {value}")

## Instruction Categories

We can group instructions by type:

| Category | Instructions | Format |
|----------|-------------|--------|
| R-type (register) | ADD, SUB, AND, OR, XOR | Rd, Rs1, Rs2 |
| R-type (unary) | NOT, SHL, SHR, MOV | Rd, Rs |
| I-type (memory) | LOAD, STORE | Rd, addr |
| J-type (jump) | JMP, JZ, JNZ | addr |
| N-type (no operands) | NOP, HALT | - |

In [ ]:
# Example: encode a small program
program = [
    encode_instruction('LOAD', rd=1, rs2_imm=0),   # LOAD R1, 0  (will load from addr 0)
    encode_instruction('LOAD', rd=2, rs2_imm=1),   # LOAD R2, 1
    encode_instruction('ADD', rd=0, rs1=1, rs2_imm=2),  # ADD R0, R1, R2
    encode_instruction('HALT'),                     # HALT
]

print("Encoded program:")
for i, instr in enumerate(program):
    decoded = decode_instruction(instr)
    val = bits_to_int_n(instr)
    print(f"  {i*2:02X}: {val:04X}  {decoded['opcode_name']}")

## Copy Your Implementation

Once your code works, copy the functions and constants to `src/computer/isa.py`

## Validation

In [ ]:
from utils.checker import check
check('isa')

## Summary

We've defined our computer's instruction set:

| Feature | Specification |
|---------|---------------|
| Instruction width | 16 bits |
| Opcode width | 4 bits (16 instructions) |
| Registers | 8 (R0-R7) |
| Address width | 8 bits (0-255 for full memory access) |
| R-type operands | 4 bits each (rd, rs1, rs2) |

### Instruction Types

| Type | Instructions | Address/Immediate |
|------|--------------|-------------------|
| R-type | ADD, SUB, AND, OR, XOR, NOT, SHL, SHR, MOV | 4-bit register |
| I-type | LOAD, STORE | 8-bit address |
| J-type | JMP, JZ, JNZ | 8-bit address |
| N-type | NOP, HALT | None |

### Key Concepts

1. **ISA** defines the CPU's language
2. **Opcodes** specify operations
3. **Operands** specify what data to use
4. **Encoding** converts assembly to binary
5. **Different formats** allow flexible addressing

### What's Next?

Now we need to build the **Instruction Decoder** - the hardware that extracts fields from instructions!